<a href="https://colab.research.google.com/github/arianmo477/EgoCentricVisionPolito/blob/main/Extension/Test_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge-score
!pip install evaluate
!pip install bert_score
!pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bac4a0e33d9707e0f23790ad951af0861c509d00e6272f5fcf13155f439d733a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import evaluate
import torch
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def evaluate_video_qa_metrics(answers_df):
    """
    Evaluates a QA model's answers using BLEU, ROUGE, and BERTScore.
    Prints all metrics directly. Does not save to file.
    """

    # Drop rows with missing values
    answers_df = answers_df.dropna(subset=['ground truth', 'answer'])

    # --- Tokenizer ---
    def simple_tokenizer(text):
        return re.findall(r'\b\w+\b', str(text).lower())

    # --- Tokenize ---
    references = [[simple_tokenizer(ref)] for ref in answers_df['ground truth']]
    hypotheses = [simple_tokenizer(pred) for pred in answers_df['answer']]

    # --- BLEU Scores ---
    smoothie = SmoothingFunction().method4
    bleu1 = corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0), smoothing_function=smoothie)
    bleu2 = corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
    bleu3 = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
    bleu4 = corpus_bleu(references, hypotheses, smoothing_function=smoothie)

    print(f"\n🔵 BLEU Scores:")
    print(f"BLEU-1: {bleu1:.4f}")
    print(f"BLEU-2: {bleu2:.4f}")
    print(f"BLEU-3: {bleu3:.4f}")
    print(f"BLEU-4: {bleu4:.4f}")

    # --- Prepare text for ROUGE & BERTScore ---
    joined_hypotheses = [' '.join(tokens) for tokens in hypotheses]
    joined_references = [' '.join(ref[0]) for ref in references]

    # --- ROUGE ---
    rouge_results = {}
    try:
        rouge = evaluate.load("rouge")
        rouge_results = rouge.compute(
            predictions=joined_hypotheses,
            references=joined_references
        )
        print("\n🔴 ROUGE Scores:")
        for k, v in rouge_results.items():
            print(f"{k.upper()}: {v:.4f}")
    except Exception as e:
        print(f"\n❗ROUGE evaluation failed: {e}")

    # --- BERTScore ---
    avg_precision = avg_recall = avg_f1 = 0.0
    try:
        bertscore = evaluate.load("bertscore")
        bertscore_results = bertscore.compute(
            predictions=joined_hypotheses,
            references=joined_references,
            lang="en"
        )
        avg_precision = sum(bertscore_results['precision']) / len(bertscore_results['precision'])
        avg_recall = sum(bertscore_results['recall']) / len(bertscore_results['recall'])
        avg_f1 = sum(bertscore_results['f1']) / len(bertscore_results['f1'])

        print("\n🟢 BERTScore (Semantic Similarity):")
        print(f"Precision: {avg_precision:.4f}")
        print(f"Recall:    {avg_recall:.4f}")
        print(f"F1 Score:  {avg_f1:.4f}")
    except Exception as e:
        print(f"\n❗BERTScore evaluation failed: {e}")

In [ ]:
# Load answers DataFrame (make sure answers_df is already defined)
# Example:
import pandas as pd
answers_df1 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/Prompt_LLAVA_VIDEO.csv")
answers_df2 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/gemini_video_qa_ans_1.5_flash_3000.csv")
answers_df3 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/gemini_video_qa_ans_1.5_pro_3000.csv")
answers_df4 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/gemini_video_qa_ans_2.5_flash.csv")
answers_df5 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/gemini_video_qa_ans_2.5_pro.csv")
answers_df6 = pd.read_csv("/content/drive/MyDrive/model_2.0/model/results/gpt4o_3000tokens.csv")





In [ ]:
print("\nVIDEO_LLAVA MODEL")
evaluate_video_qa_metrics(answers_df1)

print("\nGEMINI 1.5 flash MODEL")
evaluate_video_qa_metrics(answers_df2)

print("\nGEMINI 1.5 pro MODEL")
evaluate_video_qa_metrics(answers_df3)

print("\nGEMINI 2.5 flash MODEL")
evaluate_video_qa_metrics(answers_df4)

print("\nGEMINI 2.5 pro MODEL")
evaluate_video_qa_metrics(answers_df5)


print("\nChat GPT 4o MODEL")
evaluate_video_qa_metrics(answers_df6)


VIDEO_LLAVA MODEL

🔵 BLEU Scores:
BLEU-1: 0.2558
BLEU-2: 0.1607
BLEU-3: 0.0901
BLEU-4: 0.0333

🔴 ROUGE Scores:
ROUGE1: 0.2825
ROUGE2: 0.1280
ROUGEL: 0.2799
ROUGELSUM: 0.2800


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.8964
Recall:    0.8851
F1 Score:  0.8904

GEMINI 1.5 flash MODEL

🔵 BLEU Scores:
BLEU-1: 0.2684
BLEU-2: 0.1677
BLEU-3: 0.1023
BLEU-4: 0.0419

🔴 ROUGE Scores:
ROUGE1: 0.2852
ROUGE2: 0.1047
ROUGEL: 0.2819
ROUGELSUM: 0.2798


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.8871
Recall:    0.8923
F1 Score:  0.8893

GEMINI 1.5 pro MODEL

🔵 BLEU Scores:
BLEU-1: 0.3115
BLEU-2: 0.1992
BLEU-3: 0.1214
BLEU-4: 0.0524

🔴 ROUGE Scores:
ROUGE1: 0.3449
ROUGE2: 0.1373
ROUGEL: 0.3439
ROUGELSUM: 0.3456


...


🟢 BERTScore (Semantic Similarity):
Precision: 0.8859
Recall:    0.8910
F1 Score:  0.8880

GEMINI 2.5 pro MODEL

🔵 BLEU Scores:
BLEU-1: 0.3256
BLEU-2: 0.2058
BLEU-3: 0.1191
BLEU-4: 0.0507

🔴 ROUGE Scores:
ROUGE1: 0.3420
ROUGE2: 0.1442
ROUGEL: 0.3359
ROUGELSUM: 0.3374


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.8938
Recall:    0.8975
F1 Score:  0.8952

Chat GPT 4o MODEL

🔵 BLEU Scores:
BLEU-1: 0.2785
BLEU-2: 0.1691
BLEU-3: 0.0955
BLEU-4: 0.0434

🔴 ROUGE Scores:
ROUGE1: 0.3073
ROUGE2: 0.1047
ROUGEL: 0.3030
ROUGELSUM: 0.3031


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.8824
Recall:    0.8952
F1 Score:  0.8883
